"""
Task A: Supplier Data Cleaning and Joining
Vanilla Steel Junior Data Scientist Assessment

This script cleans and merges two supplier datasets into a unified inventory dataset.
"""

In [94]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

In [95]:
def load_data():
    """Load both supplier datasets from Excel files."""
    # Define paths
    resources_dir = Path("../../resources/task_1")
    
    # Load data
    supplier1_df = pd.read_excel(resources_dir / "supplier_data1.xlsx")
    supplier2_df = pd.read_excel(resources_dir / "supplier_data2.xlsx")
    
    return supplier1_df, supplier2_df

In [96]:
def clean_supplier1(df):
    """
    Clean and standardize supplier_data1.xlsx
    
    Assumptions:
    - Quality/Choice: Standardize to numeric (1, 2, 3) from (1st, 2nd, 3rd)
    - Grade: Standardize format by removing extra spaces and converting to uppercase
    - Finish: Standardize German terms to English equivalents
    - Thickness and Width: Already in mm, ensure float format
    - Description: Clean and standardize defect descriptions
    - Gross weight: Rename to Weight for consistency with supplier2
    - RP02, RM, AG, AI: Technical properties, keep as is but handle zeros
    - Quantity: Keep as is, handling any missing values
    """
    
    df_clean = df.copy()
    
    # Standardize Quality/Choice to numeric
    quality_map = {'1st': 1, '2nd': 2, '3rd': 3}
    df_clean['Quality'] = df_clean['Quality/Choice'].map(quality_map)
    df_clean.drop('Quality/Choice', axis=1, inplace=True)
    
    # Standardize Grade (uppercase, trim)
    df_clean['Grade'] = df_clean['Grade'].str.strip().str.upper()
    
    # Standardize Finish (translate German to English)
    finish_map = {
        'ungebeizt': 'unpickled',
        'gebeizt': 'pickled', 
        'gebeizt und geglüht': 'pickled and annealed',
        'geglüht': 'annealed'
    }
    df_clean['Finish'] = df_clean['Finish'].replace(finish_map)

    # Standardize Description (translate German to English)
    description_map = {
    'Kantenfehler - FS-Kantenrisse': 'Edge defect – FS edge cracks',
    'Längs- oder Querisse': 'Longitudinal or transverse cracks',
    'Sollmasse (Gewicht) unterschritten': 'Target weight not reached'
    }
    df_clean['Description'] = df_clean['Description'].replace(description_map)

    # Rename columns for consistency
    rename_map = {
        'Thickness (mm)': 'Thickness_mm',
        'Width (mm)': 'Width_mm',
        'Gross weight (kg)': 'Weight_kg',
    }
    df_clean.rename(columns=rename_map, inplace=True)

    # Ensure all numeric columns are float
    numeric_cols = ['Thickness_mm', 'Width_mm', 'Weight_kg',
                    'RP02', 'RM', 'Quantity', 'AG', 'AI']
    for col in numeric_cols:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').astype(float)
    
    # Handle zeros in technical properties (0 might mean not tested/applicable)
    # Replace 0 with NaN for technical properties
    tech_props = ['RP02', 'RM', 'AG', 'AI']
    for col in tech_props:
        df_clean[col] = df_clean[col].replace(0, np.nan)
    
    # Add source identifier
    df_clean['Source'] = 'Supplier1'
    
    # Reorder columns for clarity
    column_order = ['Source', 'Grade', 'Quality', 'Finish', 'Thickness_mm', 
                   'Width_mm', 'Weight_kg', 'Quantity', 'Description',
                   'RP02', 'RM', 'AG', 'AI']
    df_clean = df_clean[column_order]
    
    return df_clean

In [97]:
def clean_supplier2(df):
    """
    Clean and standardize supplier_data2.xlsx
    
    Assumptions:
    - Material: Parse to extract base grade and coating information
    - Description: Standardize surface treatment descriptions
    - Article ID: Keep as unique identifier
    - Weight: Already in kg, consistent with supplier1
    - Quantity: Keep as is
    - Reserved: Convert to boolean (True if reserved, False if not)
    """
    
    df_clean = df.copy()
    
    # Parse Material column to extract grade and coating
    def parse_material(material):
        """Extract base grade and coating from material string."""
        if pd.isna(material):
            return None, None
        
        material = str(material).strip()
        
        # Check for coating patterns (+Z, +AZ, etc.)
        if '+' in material:
            parts = material.split('+')
            grade = parts[0].strip()
            coating = '+' + parts[1].strip()
        else:
            grade = material
            coating = None
            
        return grade, coating
    
    # Apply parsing
    df_clean[['Grade', 'Coating']] = df_clean['Material'].apply(
        lambda x: pd.Series(parse_material(x))
    )
    
    # Standardize grade format
    df_clean['Grade'] = df_clean['Grade'].str.strip().str.upper()
    
    # Standardize Description to match Finish concept
    description_map = {
        'Material is Oiled': 'oiled',
        'Material is Painted': 'painted',
        'Material is not Oiled': 'not oiled'
    }
    df_clean['Finish'] = df_clean['Description'].replace(description_map)
    
    # Convert Reserved to boolean
    df_clean['Reserved'] = df_clean['Reserved'] != 'NOT RESERVED'
    
    # Add source identifier
    df_clean['Source'] = 'Supplier2'
    
    # Standardize column names for consistency
    rename_map = {
        'Thickness (mm)': 'Thickness_mm',
        'Width (mm)': 'Width_mm',
        'Weight (kg)': 'Weight_kg'
    }
    df_clean.rename(columns=rename_map, inplace=True)
    
    # Since supplier2 doesn't have thickness/width, we'll set them as NaN
    df_clean['Thickness_mm'] = np.nan
    df_clean['Width_mm'] = np.nan
    df_clean['Quality'] = np.nan  # No quality information in supplier2
    
    # Add missing technical properties as NaN
    df_clean['RP02'] = np.nan
    df_clean['RM'] = np.nan
    df_clean['AG'] = np.nan
    df_clean['AI'] = np.nan
    
    # Reorder and select columns to match supplier1 structure
    column_order = ['Source', 'Grade', 'Quality', 'Finish',
                   'Thickness_mm', 'Width_mm', 'Weight_kg',
                   'Quantity', 'Description', 'RP02', 'RM', 'AG', 'AI',
                   'Article ID', 'Coating', 'Reserved']
    
    df_clean = df_clean[column_order]
    
    return df_clean


In [98]:
def merge_datasets(df1_clean, df2_clean):
    """
    Merge the two cleaned datasets into a unified inventory dataset.
    
    Assumptions:
    - Both datasets represent different inventory items (no duplicates between them)
    - Union operation (concatenate all rows)
    - Preserve all columns, filling NaN where data is not available
    """

    # Get unique columns from each dataset
    unique_cols1 = list(set(df1_clean.columns) - set(df2_clean.columns))
    unique_cols2 = list(set(df2_clean.columns) - set(df1_clean.columns))
    
    # Add missing columns to each dataframe with NaN values
    for col in unique_cols2:
        df1_clean[col] = np.nan
    
    for col in unique_cols1:
        df2_clean[col] = np.nan
    
    # Ensure same column order
    all_columns = ['Source', 'Grade', 'Quality', 'Finish', 'Thickness_mm', 
                  'Width_mm', 'Weight_kg', 'Quantity', 'Description',
                  'RP02', 'RM', 'AG', 'AI', 'Article ID', 'Coating', 'Reserved']
    
    df1_clean = df1_clean[all_columns]
    df2_clean = df2_clean[all_columns]
    
    # Concatenate the datasets
    inventory_df = pd.concat([df1_clean, df2_clean], ignore_index=True)
    
    # Add a unique inventory ID
    inventory_df.insert(0, 'Inventory_ID', range(1, len(inventory_df) + 1))
    
    # Sort by Source and Grade for better organization
    inventory_df = inventory_df.sort_values(['Source', 'Grade'], ignore_index=True)
    
    # Update Inventory_ID after sorting
    inventory_df['Inventory_ID'] = range(1, len(inventory_df) + 1)
    
    return inventory_df

In [99]:
def main():
    """Main execution function."""
    
    # Load data
    supplier1_df, supplier2_df = load_data()
    
    # Cleaning, normalizing, and handling missing values
    supplier1_clean = clean_supplier1(supplier1_df)
    supplier2_clean = clean_supplier2(supplier2_df)
    
    # Merge datasets
    print("\nMerging datasets...")
    inventory_df = merge_datasets(supplier1_clean, supplier2_clean)
    print(f"Unified inventory dataset created: {inventory_df.shape}")  

    # Check datatypes
    print("\nColumn datatypes:")
    print(inventory_df.dtypes)
    
    # Save to CSV
    output_path = Path("../../results/inventory_dataset.csv")
    inventory_df.to_csv(output_path, index=False)
    print(f"\nInventory dataset saved to: {output_path}")

In [100]:
if __name__ == "__main__":
    inventory_df = main()


Merging datasets...
Unified inventory dataset created: (100, 17)

Column datatypes:
Inventory_ID      int64
Source           object
Grade            object
Quality         float64
Finish           object
Thickness_mm    float64
Width_mm        float64
Weight_kg       float64
Quantity        float64
Description      object
RP02            float64
RM              float64
AG              float64
AI              float64
Article ID      float64
Coating          object
Reserved         object
dtype: object

Inventory dataset saved to: ../../results/inventory_dataset.csv
